# Compliance zone for two MBS panels
**Author**: Dr Vitas Anderson (*Two Fields Consulting*)

**Last updated**: 3/11/2022

+ FEKO data provided by Danie Ludick
+ Uses **v11** RFcalcUC module
+ This notebook uses dataframes in HDF5 files that were generated from the `make_HDF5_MBS_panel_trial2.ipynb` notebook

## Load necessary python modules

In [1]:
# import mayavi mlab
from traits.etsconfig.api import ETSConfig
ETSConfig.toolkit = 'qt4'  # the 'qt4' option actually denotes qt4 and qt5
from mayavi import mlab

# import other modules
from importlib import reload
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
import pandas as pd
import RFcalcUC_v11 as RFcalcUC
from matplotlib.ticker import StrMethodFormatter
pd.set_option('display.max_rows', 200)
from icecream import ic

# Set plotting environment
sn.set()

## Create the RFc object for the analyses
The class for this object is in the `RFcalcUC_v11.py` file

In [2]:
# Read in "calc uc trials.xls" spreadsheet
trials = pd.read_excel('../antennas/spat avg trials.xlsx', 'trials', 
                       skiprows=1, index_col=0).fillna('')
# trials.drop("unit", axis=1, inplace=True)

# Set trial number
trial = 2
t = trials[str(trial)]

# Set data and plot folders
datadir = f'../antennas/{t.antenna_folder}/data/'
plotdir = f'../antennas/{t.antenna_folder}/plots/'

# Specify the RFc object parameters
freq = t.fMHz   # 900 MHz
power = t.Prad  # 80 W

# Field point grid
grid = dict(x=[t.xstart, t.xend, t.dx], 
            y=[t.ystart, t.yend, t.dy],
            z=[t.zstart, t.zend, t.dz])

# Antenna box dimensions (xyz extents)
xb = [-0.04, 0]
yb = [-0.15, 0.15]
zb = [-1.125, 1.125]
antennabox = [xb, yb, zb]

# Create the RFc object
reload(RFcalcUC)
R = RFcalcUC.RFc(freq, power, grid, antennabox, spatavgL=1.6)

# Import the S data files for FEKO and IXUS
Sfile = datadir + f'{t.antenna_folder}.hdf5'
R.importS(Sfile)

# Create filters for valid 1.6m and 2m spatial averaging points
fsa16 = R.sf('spatavg', spatavgL=1.6) # valid 1.6m spatial averaging points
foutant = R.sf('outant', offset=0)    # all points outside antenna box

# Display the first few records in the S dataframe
display(R)
display(R.S.head())

Sfile='../antennas/MBSpanel_2/data/MBSpanel_2.hdf5'


Object parameters:
  900 MHz, 200 W
  errtol = 0.15
  offset = 0 m
  default length of spatial averagering window = 1.6 m
  1,298,751 grid points
  nx = 151, ny = 141, nz = 61

,x,y,z,r,phi,SE,SH,Smax,SARwb,SARwbi
0,-1.0,-7.0,-3.0,7.071068,-98.130102,0.007138,0.007219,0.007219,NaN,NaN
1,-1.0,-7.0,-2.9,7.071068,-98.130102,0.007635,0.007725,0.007725,NaN,NaN
2,-1.0,-7.0,-2.8,7.071068,-98.130102,0.007646,0.007729,0.007729,NaN,NaN
3,-1.0,-7.0,-2.7,7.071068,-98.130102,0.007189,0.007244,0.007244,NaN,NaN
4,-1.0,-7.0,-2.6,7.071068,-98.130102,0.006404,0.006415,0.006415,NaN,NaN


In [3]:
R.grid

{'x': [-1, 14, 0.1], 'y': [-7, 7, 0.1], 'z': [-3, 3, 0.1]}

## Create spatially averaged fields
Create some new S columns for vertical spatial averaging

**c** is the convolution vector applied to z columns of points for each (x,y) group

In [4]:
def conv_vector(l, dl, m):
    '''Create a convolution vector
        l = length of convolution vector (m)
       dl = increment spacing between points in convolution vector (m)
        m = number of averaging points in convolution vector'''

    n = int(round(l/dl)) + 1  # number of points in convolution vector'''
    c = np.ones(n)            # convolution vector
    ix = np.arange(n)         # index of convolution vector
    dix = (n-1)/(m-1)         # index spacing between convolution averaging points

    assert round(l/dl,0) == round(l/dl,1), f'dl ({dl}) does not divide equally into l ({l})'
    assert type(m) == int, f'm ({m}) must be an integer'
    assert (n-1) % (m-1) == 0, f'm ({m}) does not distribute evenly across number of convolution points ({n})'

    c[(ix % dix) != 0] = 0
    c = c/m

    assert np.isclose(sum(c), 1, rtol=1e-08), f'sum of c ({sum(c)}) does not add up to 1'
    
    return c

# 1.6m vertical average curves
l, dl = 1.6, 0.1
for m in [17,5]:
    c = conv_vector(l, dl, m)
    print(f'\n{m:2d} points: sum{list(c.round(3))} = {sum(c)}')
    R.spatavg(f"Smax-{l}m-{m}",  "Smax",  c, title=f"FEKO Smax levels ({m} points averaged over {l}m)")

R.datatitles


17 points: sum[0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059] = 1.0
creating Smax-1.6m-17 spatial average of Smax

 5 points: sum[0.2, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.2] = 1.0
creating Smax-1.6m-5 spatial average of Smax


{'SE': 'S(E) levels',
 'SH': 'S(H) levels',
 'Smax': 'Smax levels',
 'SARps': 'peak spatial SAR',
 'SARwb': 'WBA SAR',
 'SARwbi': 'WBA SAR (interpolated)',
 'Smax-1.6m-17': 'FEKO Smax levels (17 points averaged over 1.6m)',
 'Smax-1.6m-5': 'FEKO Smax levels (5 points averaged over 1.6m)'}

## View public and occ compliance zones

In [5]:
# 3D view of public and occ exclusions zones (200W)
# Note how parameters for multiple exclusion zones are supplied in lists []
R.ExclusionZone(
    data=["SE", "SE"],
    power=[200, 200],
    color=["gold", "red"],
    alpha=[0.5, 0.8],
    setting=["public", "occupational"],
    standard=["RPS S-1 WB"] * 2,
    title="RPS S-1 public and occupational exclusion zones for FEKO SE data",
    figsize=(1200,800),
    axv=[True,False,True]
)

dat='SE', con=4.5
dat='SE', con=22.5
power=200, plotpower=200, setting=public, limit=4.5 W/m², contour level=4.500
power=200, plotpower=200, setting=occupational, limit=22.5 W/m², contour level=22.500


In [6]:
# Animated 3D view of public and occ exclusions zones (200W)
# Note how parameters for multiple exclusion zones are supplied in lists []
R.AnimatedExclusionZone(
    data=["SE", "SE"],
    power=[50, 50],
    color=["gold", "red"],
    alpha=[0.5, 0.5],
    setting=["public", "occupational"],
    standard=["RPS S-1 WB"] * 2,
    ycut=0,
    daz=-0.1,
    elevation = 80,
    distance=None,
    title="",
    figsize=(1200,800),
    antenna=RFcalcUC.panelAntenna,
    axv=[False,False,False]
)

dat='SE', con=18.0
dat='SE', con=90.0
power=200, plotpower=50, setting=public, limit=4.5 W/m², contour level=18.000
power=200, plotpower=50, setting=occupational, limit=22.5 W/m², contour level=90.000


In [5]:
# Show Ssa 1.6m 17pts for GitBook image
n = 1
R.ExclusionZone(
    data=["Smax-1.6m-17"],
    power=[80] * n,
    color=["blue"] * n,
    alpha=[0.4] * n,
    setting=["public"] * n,
    standard=["RPS S-1 WB"] * n,
    title="Gitbook image of compliance zone",
    ycut=None,
    axv=(False,False,False),
    figsize=(1200,800),
    bg='white'
)

dat='Smax-1.6m-17', con=11.25
power=200, plotpower=80, setting=public, limit=4.5 W/m², contour level=11.250


In [4]:
# Show Smax for GitBook image
n = 1
R.ExclusionZone(
    data=["Smax"],
    power=[80] * n,
    color=["lightgrey"] * n,
    alpha=[0.5] * n,
    setting=["public"] * n,
    standard=["RPS S-1 WB"] * n,
    title="Gitbook image of compliance zone",
    ycut=None,
    axv=(False,False,False),
    figsize=(1200,800),
    bg='white'
)

dat='Smax', con=11.25
power=200, plotpower=80, setting=public, limit=4.5 W/m², contour level=11.250


## Adjust R to combine two antennas

In [29]:
# Set vertical distance between antennas
antdz = 3  # distance between antenna centres
p2 = 0.6   # power proportion of p2 wrt p1, i.e. p2/p1 

# Create R1 as deep copy of R
import copy
R1 = copy.deepcopy(R)
dropcols = ['SE','SH','SARwb','phi','r']
R1.S.drop(columns=dropcols,inplace=True)

# Create R2 as deepy copy of R1 and offset z by antdz
R2 = copy.deepcopy(R1)
R2.S['z'] = R2.S.z + antdz

# Ensure compatibility of R2.S.z with R1.S.z for later merge operation
R2.S['z'] = R2.S.z.round(1).astype(float)

# Get the field data columns
datcols = R1.S.loc[:,"Smax":].columns.tolist()

# merge S for R1 and R2 and fill nulls with 0
S2 = pd.merge(R1.S,R2.S,'outer',on=['x','y','z'],suffixes=('_1','_2')).sort_values(['x','y','z'])
S2 = S2.fillna(0)

# Sum field levels of merged datcols in S2
for col in datcols:
    col1 = col + '_1'
    col2 = col + '_2'
    S2[col] = S2[col1] + p2 * S2[col2]

# Sort S2 by x,y,z
S2 = S2.sort_values(['x','y','z'])
# display(S2[S2.Smax.isnull()])
    
# Get rid of  _1 and _2 columns
cols = [col for col in S2.columns if '_' not in col]
S2 = S2[cols]

# Update R2 with S2 and other self variables relevant to the new enlarged S dataframe
R2.S = S2
R2.nz = len(S2.z.unique())
R2.grid = dict(x=[t.xstart, t.xend, t.dx], 
               y=[t.ystart, t.yend, t.dy],
               z=[t.zstart, t.zend+antdz, t.dz])
R2.xm = R2.make_mgrid('x')
R2.ym = R2.make_mgrid('y')
R2.zm = R2.make_mgrid('z')

R2.S.head()

,x,y,z,Smax,SARwbi,Smax-1.6m-17,Smax-1.6m-5
0,-1.0,-7.0,-3.0,0.007219,0.0,0.003416,0.003954
1,-1.0,-7.0,-2.9,0.007725,0.0,0.003911,0.004339
2,-1.0,-7.0,-2.8,0.007729,0.0,0.004615,0.004934
3,-1.0,-7.0,-2.7,0.007244,0.0,0.005593,0.005788
4,-1.0,-7.0,-2.6,0.006415,0.0,0.006896,0.008386


In [35]:
R2.ExclusionZone(
    data=["Smax-1.6m-17"],
    power=[200],
    color=["blue"],
    alpha=[0.15],
    setting=["public"],
    standard=["RPS S-1 WB"],
    title="",
    figsize=(1200,800),
    bg='white',
    axv=[False,False,False],
    antdz=antdz
)

dat='Smax-1.6m-17', con=4.5
power=200, plotpower=200, setting=public, limit=4.5 W/m², contour level=4.500


In [27]:
R2.ExclusionZone(
    data=["SARwbi", "Smax-1.6m-17"],
    power=[200, 200],
    color=["magenta", "blue"],
    alpha=[.6, 0.5],
    setting=["public", "public"],
    standard=["RPS S-1 WB"] * 2,
    title="",
    figsize=(1200,800),
    axv=[True,False,True],
    antdz=antdz
)

dat='SARwbi', con=0.08
dat='Smax-1.6m-17', con=4.5
power=200, plotpower=200, setting=public, limit=0.08 W/kg, contour level=0.080
power=200, plotpower=200, setting=public, limit=4.5 W/m², contour level=4.500


## Scratch

In [78]:
df = pd.DataFrame(np.arange(12).reshape(4,3),columns=['a','b','c'])
df.loc[1,'b'] = np.nan
df

,a,b,c
0,0,1.0,2
1,3,NaN,5
2,6,7.0,8
3,9,10.0,11


In [81]:
df[df.b.isnull()]

,a,b,c
1,3,NaN,5
